In [1]:
import torch
from einops import rearrange
import torch.nn.functional as F

In [8]:
a = torch.randint(2,10,(3,256,256))
a

tensor([[[5, 7, 3,  ..., 8, 2, 5],
         [2, 4, 2,  ..., 2, 2, 5],
         [7, 5, 2,  ..., 2, 2, 3],
         ...,
         [4, 3, 6,  ..., 3, 7, 8],
         [9, 6, 9,  ..., 7, 7, 4],
         [8, 2, 5,  ..., 4, 8, 3]],

        [[6, 6, 8,  ..., 2, 2, 9],
         [5, 2, 9,  ..., 6, 5, 6],
         [9, 9, 5,  ..., 3, 5, 7],
         ...,
         [7, 7, 5,  ..., 3, 7, 7],
         [2, 9, 7,  ..., 6, 9, 9],
         [4, 8, 6,  ..., 4, 4, 7]],

        [[7, 5, 6,  ..., 3, 8, 3],
         [6, 2, 6,  ..., 4, 3, 3],
         [9, 5, 7,  ..., 6, 8, 6],
         ...,
         [6, 8, 5,  ..., 7, 3, 8],
         [7, 8, 6,  ..., 5, 4, 2],
         [7, 6, 3,  ..., 6, 3, 5]]])

In [9]:
a.view(3,16,16,256)

tensor([[[[5, 7, 3,  ..., 8, 2, 5],
          [2, 4, 2,  ..., 2, 2, 5],
          [7, 5, 2,  ..., 2, 2, 3],
          ...,
          [6, 2, 8,  ..., 8, 3, 3],
          [9, 4, 3,  ..., 7, 9, 2],
          [4, 7, 6,  ..., 2, 4, 5]],

         [[5, 8, 7,  ..., 9, 4, 8],
          [6, 7, 7,  ..., 4, 9, 3],
          [6, 8, 3,  ..., 3, 4, 7],
          ...,
          [6, 3, 9,  ..., 6, 7, 9],
          [2, 5, 8,  ..., 3, 8, 9],
          [4, 7, 4,  ..., 2, 7, 7]],

         [[2, 9, 8,  ..., 3, 5, 9],
          [2, 4, 3,  ..., 2, 9, 2],
          [4, 6, 7,  ..., 8, 8, 9],
          ...,
          [3, 5, 8,  ..., 7, 8, 8],
          [5, 9, 8,  ..., 4, 2, 6],
          [6, 7, 4,  ..., 3, 7, 8]],

         ...,

         [[8, 4, 4,  ..., 8, 9, 4],
          [2, 8, 3,  ..., 5, 6, 4],
          [6, 6, 8,  ..., 2, 7, 4],
          ...,
          [3, 3, 4,  ..., 2, 2, 8],
          [9, 2, 8,  ..., 8, 5, 5],
          [9, 4, 7,  ..., 8, 2, 9]],

         [[4, 2, 8,  ..., 7, 9, 5],
          [2, 2, 

In [1]:
!pip install einops

In [6]:
B = 64
T = 30
P = 256
D = 768

sim = torch.rand(B,T,P).to('cuda')
l = torch.rand(B,T,D).to('cuda')

In [51]:
a = torch.rand(B,T,D).to('cuda')
b = torch.rand(B,T,D).to('cuda')
mask = torch.eye(B,T).to('cuda')

In [52]:
def masked_pairwise_contrastive_loss(a, b, mask):
    batch_size, seq_len, _ = a.shape
    mask_logits = ~mask.bool().unsqueeze(-1).repeat(1,1,seq_len)
    print(mask_logits.shape)
    labels = torch.eye(seq_len).repeat(batch_size,1,1).to(a.device)
    logits = torch.einsum('bmd,bnd->bmn',a,b)
    INF = torch.finfo(logits.dtype).min
    logits = logits.masked_fill(mask_logits, INF)
    
    # flatten all vectors
    logits = torch.flatten(logits, 0, 1)
    labels = torch.flatten(labels, 0, 1)
    mask_logits = torch.flatten(mask_logits, 0, 1)
    print(mask_logits)
    
    # loss
    log_prob = F.log_softmax(logits, dim=-1)
    loss = -log_prob*labels
    print(loss)
    loss = loss[mask_logits].mean()
    return loss

In [53]:
masked_pairwise_contrastive_loss(a,b,mask)

torch.Size([64, 30, 30])
tensor([[False, False, False,  ..., False, False, False],
        [ True,  True,  True,  ...,  True,  True,  True],
        [ True,  True,  True,  ...,  True,  True,  True],
        ...,
        [ True,  True,  True,  ...,  True,  True,  True],
        [ True,  True,  True,  ...,  True,  True,  True],
        [ True,  True,  True,  ...,  True,  True,  True]], device='cuda:0')
tensor([[3.9743, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 3.4012, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 3.4012,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 3.4012, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 3.4012, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 3.4012]],
       device='cuda:0')


tensor(0.1134, device='cuda:0')

In [15]:
sim = sim/(torch.max(sim, dim=-1).values.unsqueeze(-1) - torch.min(sim, dim=-1).values.unsqueeze(-1))
sim.shape

torch.Size([64, 30, 256])

In [20]:
sim/torch.sum(sim, dim=-1).unsqueeze(-1)

tensor([[[5.7560e-03, 2.7589e-03, 4.8367e-04,  ..., 2.7247e-03,
          5.6856e-03, 7.2506e-03],
         [7.4191e-04, 1.0250e-05, 3.9257e-03,  ..., 1.5827e-03,
          2.9699e-03, 6.9620e-03],
         [5.4153e-03, 4.9174e-03, 4.3338e-03,  ..., 1.7716e-03,
          5.0803e-03, 4.6817e-03],
         ...,
         [4.8410e-03, 6.4017e-03, 6.9732e-03,  ..., 6.1932e-03,
          5.9526e-03, 3.9711e-03],
         [7.5591e-03, 1.7085e-03, 5.4239e-03,  ..., 4.9801e-03,
          4.5435e-03, 6.6362e-03],
         [2.7200e-03, 6.9640e-03, 1.8543e-03,  ..., 1.8166e-03,
          1.9588e-04, 3.8905e-04]],

        [[1.3036e-03, 1.2784e-03, 7.7186e-03,  ..., 2.7483e-03,
          1.9372e-03, 7.9499e-03],
         [5.8103e-03, 6.6702e-03, 4.1411e-03,  ..., 4.6495e-03,
          5.6949e-03, 4.9467e-03],
         [5.7349e-03, 1.8169e-03, 4.7077e-04,  ..., 1.5114e-03,
          9.0372e-04, 3.8160e-03],
         ...,
         [5.8712e-03, 2.6464e-03, 6.2165e-03,  ..., 1.7041e-03,
          6.049

In [129]:
l_token_embed = torch.rand(64, 34, 768).to('cuda')
v_patch_embed = torch.rand(64, 256, 768).to('cuda')
language_mask = torch.rand(64,34).to('cuda')

In [130]:
similarity = torch.einsum('btd,bpd->btp',l_token_embed, v_patch_embed)
similarity.shape

torch.Size([64, 34, 256])

In [131]:
similarity = (similarity-torch.min(similarity, dim=-1).values.unsqueeze(-1))/(torch.max(similarity, dim=-1).values - torch.min(similarity, dim=-1).values).unsqueeze(-1)

In [132]:
similarity = torch.where(similarity < 0.5, 0.0, similarity)

In [133]:
v_align_weights = similarity/torch.sum(similarity, dim=-1).unsqueeze(-1)
l_grouped_v_patch_embed = torch.einsum('btp,bpd->btd', v_align_weights, v_patch_embed)
l_grouped_v_patch_embed.shape

torch.Size([64, 34, 768])

In [134]:
mask_logits = (1.0-language_mask).unsqueeze(1).expand(-1, l_token_embed.shape[1], -1)

In [135]:
sim_token_targets = torch.eye(l_token_embed.shape[1]).unsqueeze(0).expand(l_token_embed.shape[0], -1, -1).to('cuda')

In [136]:
sim_token_i2t = torch.einsum('bmd,bnd->bmn', l_grouped_v_patch_embed, l_token_embed)
sim_token_t2i = torch.einsum('bmd,bnd->bmn', l_token_embed, l_grouped_v_patch_embed)

In [137]:
sim_token_i2t = torch.flatten(sim_token_i2t, start_dim=0, end_dim=1)
sim_token_t2i = torch.flatten(sim_token_t2i, start_dim=0, end_dim=1)
sim_token_targets = torch.flatten(sim_token_targets, start_dim=0, end_dim=1)

In [138]:
sim_token_i2t1

tensor([[193.3438, 187.4263, 193.9430,  ..., 200.3852, 193.4124, 196.0533],
        [192.4890, 188.4600, 194.1176,  ..., 200.2485, 193.5657, 196.0668],
        [192.7252, 187.8056, 195.6976,  ..., 200.6323, 193.8745, 196.2714],
        ...,
        [203.8492, 191.7299, 197.6203,  ..., 199.5779, 201.3323, 200.3034],
        [203.6092, 191.7253, 197.4860,  ..., 198.6888, 201.8735, 200.2687],
        [203.4427, 191.5200, 197.1854,  ..., 198.2348, 200.8883, 200.6276]])

In [139]:
mask_logits = torch.flatten(mask_logits, start_dim=0, end_dim=1)
loss_token_i2t = -torch.nn.functional.log_softmax(sim_token_i2t - mask_logits * float('inf'), dim=-1)*sim_token_targets
loss_token_t2i = -torch.nn.functional.log_softmax(sim_token_t2i - mask_logits * float('inf'), dim=-1)*sim_token_targets

In [140]:
loss_token_i2t.shape

torch.Size([2176, 34])

In [143]:
(1-mask_logits).shape

torch.Size([2176, 34])

In [111]:
import torch
import torch.nn.functional as F

# Assuming logits, mask_logits, and labels are your input tensors

# Set elements of mask_logits to negative infinity where mask is True
mask = (mask_logits == 1)
sim_token_i2t[mask] = float('-inf')

# Compute softmax cross-entropy loss
loss = F.cross_entropy(sim_token_i2t, sim_token_targets)

In [112]:
loss

tensor(8.3092, device='cuda:0')

In [95]:
torch.nn.functional.log_softmax(sim_token_i2t - mask_logits * float('inf'), dim=-1)*sim_token_targets

RuntimeError: The size of tensor a (34) must match the size of tensor b (73984) at non-singleton dimension 1

In [109]:
print(torch.nn.functional.log_softmax(sim_token_i2t + (mask_logits + 1e-45).log(), dim=-1))

tensor([[ -1.6944, -16.1906,  -7.4569,  ...,  -8.1323, -11.3844,  -8.9391],
        [ -2.2487, -15.0631,  -7.3145,  ...,  -8.1771, -11.2736,  -8.5244],
        [ -2.3877, -16.1607,  -6.1608,  ...,  -8.1342, -11.1276,  -8.7727],
        ...,
        [ -6.2352, -10.0961,  -3.1049,  ...,  -1.0768,  -7.6882,  -6.7549],
        [ -5.8312,  -9.6440,  -2.7862,  ...,  -1.9446,  -6.5280,  -7.0041],
        [ -6.1809,  -9.8137,  -2.5071,  ...,  -2.0764,  -7.6193,  -5.6642]],
       device='cuda:0')


In [64]:
loss

torch.Size([64, 34, 34])

In [ ]:
similarity = (similarity-torch.min(similarity, dim=-1).values)/(torch.max(similarity, dim=-1).values - torch.min(similarity, dim=-1).values)
similarity = torch.where(similarity < self.similarity_threshold, 0.0, similarity)
v_align_weights = similarity/sum(similarity, dim=-1)
l_grouped_v_patch_embed = torch.einsum('btp,bpd->btd', v_align_weights, v_patch_embed)
l_grouped_v_patch_embed = F.normalize(l_grouped_v_patch_embed, dim=-1)
l_token_embed = F.normalize(l_token_embed, dim=-1)
mask_logits = (1.0-language_mask).unsqeeze(1).expand(-1, l_token_embed.shape[1], -1)
sim_token_targets = torch.eye(l_token_embed.shape[1]).unsqeeze(0).expand(l_token_embed.shape[0], -1, -1)
sim_token_i2t = torch.einsum('bmd,bnd->bmn', l_grouped_v_patch_embed, l_token_embed) / self.temp_token
sim_token_t2i = torch.einsum('bmd,bnd->bmn', l_token_embed, l_grouped_v_patch_embed) / self.temp_token
sim_token_i2t = torch.flatten(sim_token_i2t, start_dim=0, end_dim=1)
sim_token_t2i = torch.flatten(sim_token_t2i, start_dim=0, end_dim=1)
sim_token_targets = torch.flatten(sim_token_targets, start_dim=0, end_dim=1)
mask_logits = torch.flatten(mask_logits, start_dim=0, end_dim=1)
loss_token_i2t = torch.sum(-torch.sum(F.log_softmax(sim_token_i2t - mask_logits * float('inf'), dim=-1)*sim_token_targets, dim=-1))/torch.sum(language_mask)
loss_token_t2i = torch.sum(-torch.sum(F.log_softmax(sim_token_t2i - mask_logits * float('inf'), dim=-1)*sim_token_targets, dim=-1))/torch.sum(language_mask)

In [8]:
idx = torch.tensor([2,3,4,5,7])
idx_all = torch.tensor([1,2,3,4,5,6,7,8,9,10])

In [9]:
idx = idx.view(-1,1)

In [10]:
pos_idx = torch.eq(idx, idx_all).float()
pos_idx

tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]])

In [12]:
pos_idx.sum(1,keepdim=True)

tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.]])

In [14]:
torch.full((10,1),-200)

tensor([[-200],
        [-200],
        [-200],
        [-200],
        [-200],
        [-200],
        [-200],
        [-200],
        [-200],
        [-200]])

In [15]:
idx_neg = torch.full((len(idx),1),-200)
idx_neg_queue = torch.full((len(idx),1),-300)
idx_neg_queue = torch.cat([idx, idx_neg_queue], dim=0)

In [16]:
idx_neg

tensor([[-200],
        [-200],
        [-200],
        [-200],
        [-200]])

In [17]:
idx_neg_queue

tensor([[   2],
        [   3],
        [   4],
        [   5],
        [   7],
        [-300],
        [-300],
        [-300],
        [-300],
        [-300]])

In [11]:
sim_targets = pos_idx / pos_idx.sum(1,keepdim=True)
sim_targets

tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]])

In [6]:
a = torch.tensor([[[1,1,1]],[[0,0,0]]])
b = torch.tensor([[0,0,0],[1,1,1]])
print(a.shape)
print(a)
print(b.shape)
print(b)

torch.Size([2, 1, 3])
tensor([[[1, 1, 1]],

        [[0, 0, 0]]])
torch.Size([2, 3])
tensor([[0, 0, 0],
        [1, 1, 1]])


In [7]:
c = a+b
print(c)
print(c.shape)

tensor([[[1, 1, 1],
         [2, 2, 2]],

        [[0, 0, 0],
         [1, 1, 1]]])
torch.Size([2, 2, 3])


In [9]:
d = torch.tensor([[[1, 0, 0],[0, 1, 0]],
     [[0, 0, 1],[0, 0, 0]]])
print(d.shape)

torch.Size([2, 2, 3])


In [14]:
e = torch.matmul(c.unsqueeze(1),d.permute(0,2,1))

In [25]:
e.shape #[txt, img, txt+img_dom, img+txt_dom]

torch.Size([2, 2, 2, 2])

In [17]:
d

tensor([[[1, 0, 0],
         [0, 1, 0]],

        [[0, 0, 1],
         [0, 0, 0]]])

In [22]:
print(e.shape)
f,_ = e.max(dim=-1)
print(f.shape)
g,_ = f.max(dim=-1)
print(g.shape)

torch.Size([2, 2, 2, 2])
torch.Size([2, 2, 2])
torch.Size([2, 2])


In [23]:
g

tensor([[2, 2],
        [1, 1]])

In [11]:
f = c.unsqueeze(1)
print(f.shape)

torch.Size([2, 1, 2, 3])


In [9]:
x = torch.rand(64,256)
y = torch.rand(19,256)

In [13]:
z = torch.stack([x,y], dim=0)

RuntimeError: stack expects each tensor to be equal size, but got [64, 256] at entry 0 and [19, 256] at entry 1

In [18]:
z = torch.rand(5000,5000)
topk_sim1, topk_idx1 = z.topk(k=10, dim=1)


In [19]:
topk_idx

tensor([[2762, 4839, 4376,  ..., 1090, 4061, 2072],
        [4009, 3231, 4537,  ..., 3896, 4231, 2385],
        [4648,  661, 1059,  ..., 4951, 3614, 4642],
        ...,
        [3244, 3532, 4910,  ..., 4751, 2654, 2580],
        [3426, 3192, 1365,  ...,  333, 2224, 2002],
        [  85, 1648, 2967,  ..., 4040, 1475, 4555]])

In [17]:
topk_sim.shape

torch.Size([10, 5000])

In [21]:
topk_idx1.shape

torch.Size([5000, 10])

In [24]:
s = torch.rand(5000,256)

In [25]:
s[topk_idx1].shape

torch.Size([5000, 10, 256])

In [29]:
a = s[topk_idx1[1]]
b = s[topk_idx1][1]

In [17]:
import torch
import torch.nn.functional as F
from tqdm import tqdm
a = torch.rand(5000,256)
a_dom = torch.rand(5000,256)
b = torch.rand(5000,256)
b_dom = torch.rand(5000,256)


In [18]:
sims = torch.rand(len(a),len(b))
for im in tqdm(range(len(a))):
    imn = []
    for im_d in range(len(b_dom)):
        imn.append(F.normalize(a[im] + b_dom[im_d],dim=-1))
    imn = torch.stack(imn, dim=0)
    for tx in tqdm(range(len(b))):
        txn = []
        for tx_d in range(len(a_dom)):
            txn.append(F.normalize(b[tx] + a_dom[tx_d], dim=-1))
        txn = torch.stack(txn, dim=0)
        sim = imn @ txn.t()
        sims[im,tx] = torch.max(torch.max(sim))

  0%|          | 0/5000 [00:49<?, ?it/s]


KeyboardInterrupt: 

In [21]:
'checkpoint_best_{}.pth'.format(str(5))

'checkpoint_best_5.pth'